In [7]:
# TensorFlow and tf.keras
import tensorflow as tf
import keras
import sys
sys.path.insert(0, '../')

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

1.11.0


# Sub-hypothesis 1

### Import the MNIST dataset/ Preprocess the data

In [40]:
mnist = keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [41]:
n, img_rows, img_cols = x_train.shape

In [42]:
D = img_rows * img_cols

In [43]:
x_train = x_train.reshape(n, D)
x_test = x_test.reshape(test_images.shape[0], D)

In [44]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [58]:
y_train[0:5, :]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]], dtype=float32)

### Train and test the network on MNIST

In [46]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, MaxPooling2D, Dropout, Flatten

In [47]:
from adv_util import create_fully_connected

In [48]:
input_shape = (D, )
num_classes = 10
model = create_fully_connected(input_shape = input_shape, num_classes = num_classes)
model.compile(optimizer="sgd", loss='categorical_crossentropy', metrics=['accuracy'])

In [49]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                25120     
_________________________________________________________________
activation_3 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.fit(x_train, y_train, batch_size=128, epochs=15, verbose=True, validation_split=.1)

Train on 54000 samples, validate on 6000 samples
Epoch 1/15
54000/54000 [==============================] - 1s 19us/step - loss: 1.0646 - acc: 0.5875 - val_loss: 1.0017 - val_acc: 0.5997
Epoch 2/15
54000/54000 [==============================] - 1s 18us/step - loss: 1.0514 - acc: 0.5890 - val_loss: 1.0096 - val_acc: 0.5938
Epoch 3/15
54000/54000 [==============================] - 1s 17us/step - loss: 1.0510 - acc: 0.5884 - val_loss: 0.9729 - val_acc: 0.5962
Epoch 4/15
54000/54000 [==============================] - 1s 17us/step - loss: 1.0443 - acc: 0.5855 - val_loss: 1.0278 - val_acc: 0.5863
Epoch 5/15
54000/54000 [==============================] - 1s 17us/step - loss: 1.0537 - acc: 0.5828 - val_loss: 1.0171 - val_acc: 0.6003
Epoch 6/15
54000/54000 [==============================] - 1s 18us/step - loss: 1.0279 - acc: 0.5909 - val_loss: 0.9975 - val_acc: 0.5952
Epoch 7/15
54000/54000 [==============================] - 1s 18us/step - loss: 1.0364 - acc: 0.5856 - val_loss: 1.0157 - val_acc:

In [55]:
loss, accuracy  = model.evaluate(x_test, y_test, verbose=False)

In [56]:
print(accuracy)

0.5888
